In [1]:
import pandas as pd
import numpy as np
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [2]:
imdb_df = pd.read_csv("/home/vegito/Machine_learning/DATASET/IMDB Dataset.csv")

In [ ]:
imdb_df.shape

In [ ]:
imdb_df.head(2)

In [4]:
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import re

wordnet_lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))
puctuation = string.punctuation
puctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [10]:
# classifier
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegressionCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.preprocessing import MaxAbsScaler, StandardScaler

from sklearn.pipeline import Pipeline
from pipelinehelper import PipelineHelper
from sklearn.model_selection import GridSearchCV


In [5]:
# from pandarallel import pandarallel
# pandarallel.initialize(nb_workers=4)#,progress_bar=True)


INFO: Pandarallel will run on 4 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [6]:
def clean_text(text):
    text = text.replace("<br /><br />","")
    tokens = word_tokenize(text)
    tokens =  [wordnet_lemmatizer.lemmatize(tok,pos="v") for tok in tokens if tok not in puctuation]
    tokens =  [tok.lower() for tok in tokens if tok not in stop_words]
    return tokens
    

In [7]:
tfidf = TfidfVectorizer(tokenizer = clean_text)


In [ ]:
classifier = LinearSVC()


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(imdb_df["review"], imdb_df["sentiment"], test_size = 0.2, random_state = 42)
X_train.shape, X_test.shape

In [ ]:
clf = Pipeline([('tfidf', tfidf), ('clf', classifier)])
clf.fit(X_train, y_train)

In [ ]:
y_pred = clf.predict(X_test)


In [ ]:
print(classification_report(y_test, y_pred))


In [ ]:
confusion_matrix(y_test, y_pred)


In [ ]:
clf.predict(['Worth of watching it. Please like it'])


In [ ]:
a = clf.predict(['I  hate it this thing for worst moveies'])


# GRID SEARCH

In [17]:
sgd = SGDClassifier(n_jobs=-1, random_state=42, max_iter=200)
lgr = LogisticRegression(random_state=42, max_iter=200)
lgr_cv = LogisticRegressionCV(cv=2, random_state=42, max_iter=1000)
svm = LinearSVC(random_state=42, max_iter=200)
rfc = RandomForestClassifier(random_state=42, n_jobs=-1, n_estimators=200)

In [18]:
clfn = {'SGD': sgd, 'LGR': lgr, 'LGR_CV': lgr_cv, 'SVM': svm, 'RFC': rfc}

In [25]:
def classify(X,y):
    X = tfidf.fit_transform(X)
    scaler = MaxAbsScaler()
    X = scaler.fit_transform(X)
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42, stratify = y)
    
    for key in clfn.keys():
        clfn[key].fit(X_train, y_train)
        y_pred = clfn[key].predict(X_test)
        ac = accuracy_score(y_test, y_pred)
        print(key, ' ---> ' , ac)

In [26]:
classify(imdb_df["review"], imdb_df["sentiment"])

SGD  --->  0.8972


/home/vegito/anaconda3/envs/pynlp/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LGR  --->  0.9023
LGR_CV  --->  0.9003


/home/vegito/anaconda3/envs/pynlp/lib/python3.6/site-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


SVM  --->  0.8864
RFC  --->  0.8605
